In [ ]:
!pip3 install  -r  requirements.txt

In [ ]:
#!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install  llama-cpp-python --no-cache-dir
#!pip install -q llama-index

In [4]:
#imports
from llama_index import VectorStoreIndex, SimpleDirectoryReader, ServiceContext
import pandas as pd
import json
from llama_index.llms import LlamaCPP
from llama_index.llms.llama_utils import messages_to_prompt, completion_to_prompt
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from llama_index import LangchainEmbedding, ServiceContext, Document
from llama_index import set_global_service_context

In [5]:
#Constants and global variables
s3_input_file_path ='s3://abb8-ml-model/dailydownload/mailContents.json'
s3_output_file_path ='s3://abb8-ml-model/dailydownload/insights.json'
s3_prompt_questions_path='./data/questions.json'

In [6]:
    #print(df)
    #print(df["mail_body"][1])
docs = pd.read_json(s3_input_file_path)
docs.insert(3,"insight_dates","")
docs.insert(4,"summary","")
questions=pd.read_json(s3_prompt_questions_path)
print(docs.head(1))
print(questions.head(1))

   thread_id  mail_subject                                          mail_body   
0       1001  utility_bill   Rob, We hope that you’re enjoying your electr...  \

  insight_dates summary  
0                        
                                            question
0  Can you please convert the given text material...


In [6]:
documents = SimpleDirectoryReader("./data/").load_data()
print(documents)

[Document(id_='6f7f56cc-9a36-4428-9852-4cd61fa77a53', embedding=None, metadata={'file_path': 'data/1001.txt', 'creation_date': '2023-11-09', 'last_modified_date': '2023-11-09', 'last_accessed_date': '2023-11-09'}, excluded_embed_metadata_keys=['creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, hash='7b4c211a6382104957e08c26acf5d326457a28d03a38dbd3801bb201b35b00b4', text='Hello Natasha,Just a friendly reminder about your upcoming appointment with DR. Janet Baker DDS. Please confirm your appointment by clicking the button below.____________________Your appointment is: Tuesday, December 16 2023  3:30 PM', start_char_idx=None, end_char_idx=None, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n'), Document(id_='19a8762b-c1b9-41de-a0ef-c5657e2340c3', embedding=None, metadata={'file_path': 'data/1002.txt', 'creation_date'

In [46]:
def create_insights():    
    #documents = []
    for index,row in docs.iterrows():
        documents = []
        thread_id=row["thread_id"]
        thread_text=row["mail_body"]
        thread_subject=row["mail_subject"]
        #document = Document(text=thread_text,id_=thread_id,metadata={"subject":thread_subject})
        documents.append(
            Document(text=thread_text,id_=thread_id,metadata={"subject":thread_subject})
        )
        service_context=get_service_context();
        index=VectorStoreIndex.from_documents(documents,service_context=service_context)
        query_engine = index.as_query_engine()
        for index,row in questions.iterrows():
            response = query_engine.query(row["question"])
            print('creating summary for ' + thread_subject)
            print(response)
            if index == 0:
                docs.at[index,'insight_dates']=response
            else:
                docs.at[index,'summary']=response
        #index.delete(thread_id)
    print(docs.head(1))    
    docs.to_json('s3://abb8-ml-model/dailydownload/insights.json',orient='records')        
    

In [ ]:
docs['json'] = docs.apply(lambda x: x.to_json(), axis=1)
for index,row in docs.iterrows():
    f = open('./data/'+str(row["thread_id"])+'.txt', 'w')
    f.write(row["mail_body"])
    f.close()
    documents = SimpleDirectoryReader(input_files=['./data/'+str(row["thread_id"])+'.txt']).load_data()
    #print(documents)
    print('index' + str(index))
    service_context=get_service_context();
    llamind=VectorStoreIndex.from_documents(documents,service_context=service_context)   
    
    query_engine = llamind.as_query_engine()
    print('creating summary for ' + str(row["thread_id"]))
    for index,row in questions.iterrows():        
        print(row["question"])
        response = query_engine.query(row["question"])        
        print(response)
        if index == 0:
            docs.at[index,'insight_dates']=response
        else:
            docs.at[index,'summary']=response
print(docs.head(1))    
docs.to_json('s3://abb8-ml-model/dailydownload/insights.json',orient='records')              
    

In [16]:
print(docs['json'])

0    {"thread_id":1001,"mail_subject":"utility_bill...
1    {"thread_id":1002,"mail_subject":"utility_bill...
2    {"thread_id":1001,"mail_subject":"utility_bill...
Name: json, dtype: object


In [7]:
def get_service_context():    
    llm = LlamaCPP(
        # You can pass in the URL to a GGML model to download it automatically
        model_url=None,
        # optionally, you can set the path to a pre-downloaded model instead of model_url
        model_path='./models/zephyr-7b-alpha.Q5_K_M.gguf',
        temperature=0.1,
        max_new_tokens=256,
        # llama2 has a context window of 4096 tokens, but we set it lower to allow for some wiggle room
        context_window=3900,
        # kwargs to pass to __call__()
        generate_kwargs={},
        # kwargs to pass to __init__()
        # set to at least 1 to use GPU
        model_kwargs={"n_gpu_layers": -1},
        # transform inputs into Llama2 format
        messages_to_prompt=messages_to_prompt,
        completion_to_prompt=completion_to_prompt,
        verbose=True,
    )
    embed_model = LangchainEmbedding(
      HuggingFaceEmbeddings(model_name="thenlper/gte-large")
    )
    service_context = ServiceContext.from_defaults(
        chunk_size=256,
        llm=llm,
        embed_model=embed_model
    )
    return service_context


In [ ]:
# create and set service context just once never include this in any loops
service_context=get_service_context();
#set_global_service_context(service_context)

In [22]:
index=VectorStoreIndex.from_documents(documents,service_context=service_context)

In [ ]:
print(docs.head(1)) 

In [ ]:
for index,row in docs.iterrows():
        documents = []
        thread_id=row["thread_id"]
        thread_text=row["mail_body"]
        thread_subject=row["mail_subject"]
        #document = Document(text=thread_text,id_=thread_id,metadata={"subject":thread_subject})
        documents.append(
            Document(text=thread_text,id_=thread_id,metadata={"subject":thread_subject})
        )
        print(documents)
        service_context=get_service_context();
        index=VectorStoreIndex.from_documents(documents,service_context=service_context)
        print(index.ref_doc_info)
        query_engine = index.as_query_engine()
        for index,row in questions.iterrows():
            print('question is:' +row["question"])
            response = query_engine.query(row["question"])
            print('creating summary for ' + thread_subject)
            print(response)
            if index == 0:
                docs.at[index,'insight_dates']=response
            else:
                docs.at[index,'summary']=response
        #index.delete(thread_id)
print(docs.head(1))    
docs.to_json('s3://abb8-ml-model/dailydownload/insights.json',orient='records')

In [ ]:
create_insights()
#print (documents)

In [38]:
query_engine = index.as_query_engine()

In [39]:
response = query_engine.query("Can you please provide the dates and context as json  in the given text material?")
print(response)

"
{
  "appointment": {
    "date": "Tuesday, December 16 2023",
    "time": "3:30 PM"
  },
  "context": "Hello Natasha,\nJust a friendly reminder about your upcoming appointment with DR. Janet Baker DDS.\nPlease confirm your appointment by clicking the button below."
}


In [ ]:
#from llama_index import set_global_service_context
#set_global_service_context(service_context)
print(docs)

In [20]:

print(index.index_id)
print(index.ref_doc_info)

82d2e6e9-9d2d-418d-a2f1-2184aef04695
{'6f7f56cc-9a36-4428-9852-4cd61fa77a53': RefDocInfo(node_ids=['cb424b3e-fa45-47c7-bcc0-c03d9e0a94e2'], metadata={'file_path': 'data/1001.txt', 'creation_date': '2023-11-09', 'last_modified_date': '2023-11-09', 'last_accessed_date': '2023-11-09'}), '19a8762b-c1b9-41de-a0ef-c5657e2340c3': RefDocInfo(node_ids=['46e23126-054b-495c-932f-acd0c14190a4'], metadata={'file_path': 'data/1002.txt', 'creation_date': '2023-11-09', 'last_modified_date': '2023-11-09', 'last_accessed_date': '2023-11-09'}), 'e6825e59-7fb2-4f76-9338-444a10fbadec': RefDocInfo(node_ids=['37694768-ebbe-4871-8547-6d5e036dc7a7', '12b9c48f-78dc-4cba-ae97-22532e644195'], metadata={'file_path': 'data/mailcontents.txt', 'creation_date': '2023-11-07', 'last_modified_date': '2023-11-07', 'last_accessed_date': '2023-11-07'}), 'ff0e30ac-485d-48aa-aaf9-43baac292f18': RefDocInfo(node_ids=['f3720a3f-bc55-4154-b64e-ef3e6cbd7b71'], metadata={'file_path': 'data/questions.json', 'creation_date': '2023-11

In [11]:
query_engine = index.as_query_engine()
response = query_engine.query("write a concise summary of the document id 1001 with 10 words based on the given text material.")
print(response)

SYS>>

Parent-teacher conference reminder for Isabella's family scheduled for Nov. 25th at 10:30 a.m./p.m. in room 225, lasting approximately 30 minutes. Contact school if unable to attend.


In [ ]:
#index.delete('7a91dfe9-5b66-4cdb-b5f4-dc54671e7c8a')

In [ ]:

query_engine = index.as_query_engine()
response = query_engine.query("What is the deadline to pay utility bill ?")
print(response)

In [ ]:
response = query_engine.query("Can you please convert the given text material into JSON format with the appointment dates and context included? ")
print(response)

In [8]:
service_context=get_service_context();
for index,row in docs.iterrows():
    if index >= 1:
        documents = []
        thread_id=row["thread_id"]
        thread_text=row["mail_body"]
        thread_subject=row["mail_subject"]
        #document = Document(text=thread_text,id_=thread_id,metadata={"subject":thread_subject})
        documents.append(
            Document(text=thread_text,id_=thread_id,metadata={"subject":thread_subject})
        )
        #print(documents)        
        llamaindex=VectorStoreIndex.from_documents(documents,service_context=service_context)
        ##print(index.ref_doc_info)
        query_engine = llamaindex.as_query_engine(response_mode="compact")
        response = query_engine.query('''Give me info on the following points in JSON format-
                              person
                              meeting_date
                              place
                              ''')
        docs.at[index,'summary']=response
        print(response)
        #index.delete(thread_id)
print(docs.head(2))    
docs.to_json('s3://abb8-ml-model/dailydownload/insights.json',orient='records')

llama_model_loader: loaded meta data with 21 key-value pairs and 291 tensors from ./models/zephyr-7b-alpha.Q5_K_M.gguf (version GGUF V2)
llama_model_loader: - tensor    0:                token_embd.weight q5_K     [  4096, 32000,     1,     1 ]
llama_model_loader: - tensor    1:           blk.0.attn_norm.weight f32      [  4096,     1,     1,     1 ]
llama_model_loader: - tensor    2:            blk.0.ffn_down.weight q6_K     [ 14336,  4096,     1,     1 ]
llama_model_loader: - tensor    3:            blk.0.ffn_gate.weight q5_K     [  4096, 14336,     1,     1 ]
llama_model_loader: - tensor    4:              blk.0.ffn_up.weight q5_K     [  4096, 14336,     1,     1 ]
llama_model_loader: - tensor    5:            blk.0.ffn_norm.weight f32      [  4096,     1,     1,     1 ]
llama_model_loader: - tensor    6:              blk.0.attn_k.weight q5_K     [  4096,  1024,     1,     1 ]
llama_model_loader: - tensor    7:         blk.0.attn_output.weight q5_K     [  4096,  4096,     1,     1 ]


{
  "person": "Robert Connor Family",
  "meeting_date": "11/25/2023",
  "place": "room number __225__"
}

{
  "person": "Natasha",
  "meeting_date": "Tuesday, December 16 2023",
  "place": null
}
   thread_id        mail_subject   
0       1001        utility_bill  \
1       1002  utility_bill march   

                                           mail_body insight_dates   
0   Rob, We hope that you’re enjoying your electr...                \
1  PARENT-TEACHER CONFERENCE REMINDER -springfiel...                 

                                             summary  
0                                                     
1  \n{\n  "person": "Robert Connor Family",\n  "m...  



llama_print_timings:        load time =   79044.16 ms
llama_print_timings:      sample time =      21.57 ms /    43 runs   (    0.50 ms per token,  1993.69 tokens per second)
llama_print_timings: prompt eval time =   27722.73 ms /   113 tokens (  245.33 ms per token,     4.08 tokens per second)
llama_print_timings:        eval time =   14803.14 ms /    42 runs   (  352.46 ms per token,     2.84 tokens per second)
llama_print_timings:       total time =   42665.19 ms


In [ ]:
from typing import List
from pydantic import BaseModel
from datetime import datetime, timezone
service_context=get_service_context();
index=VectorStoreIndex.from_documents(documents,service_context=service_context)
class insightInfo(BaseModel):
    person: List[str]
    meeting_date: str
    place:str

query_engine = index.as_query_engine(response_mode="compact")
response = query_engine.query('''Give me info on the following points in JSON format-
                              person
                              meeting_date
                              place
                              ''')
print(response)

In [ ]:
query_engine = index.as_query_engine()
response = query_engine.query("is there an upcoming doctor appointment for Natasha?")
print(response)

In [ ]:
query_engine = index.as_query_engine()
response = query_engine.query("is there an upcoming  parent teacher confernce for rob?")
print(response)

In [ ]:
query_engine = index.as_query_engine()
response = query_engine.query("list the upcoming tasks like doctor appointment, utility bill?")
print(response)

In [ ]:
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.document_loaders import TextLoader
from langchain.document_loaders import DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [ ]:
from langchain.document_loaders import S3DirectoryLoader

In [ ]:
loader = DirectoryLoader('./data/')
#bucket='abb8-ml-model',prefix='dailydownload'

In [ ]:
!pip install -- quite unstructured
documents =loader.load()

In [ ]:
print(documents)

In [ ]:
persist_directory = 'vectordb'
text_splitter = RecursiveCharacterTextSplitter(chunk_size=200, chunk_overlap=25)
texts = text_splitter.split_documents(documents)



In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings, SentenceTransformerEmbeddings
embeddings = HuggingFaceEmbeddings(model_name="thenlper/gte-large")
# HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

In [ ]:
vectordb = Chroma.from_documents(documents=texts,                                 
                                 embedding=embeddings,
                                 persist_directory=persist_directory)

In [ ]:
vectordb = None

In [ ]:
vectordb = Chroma(persist_directory=persist_directory, 
                  embedding_function=embeddings)

In [ ]:
qa_chain = RetrievalQA.from_chain_type(llm=llm, 
                                  chain_type="stuff", 
                                  retriever=retriever, 
                                  return_source_documents=True)

In [ ]:
retriever = vectordb.as_retriever()

In [ ]:
docs = retriever.get_relevant_documents("list the doctor appointments?")

In [ ]:
print(docs)

In [ ]:
vectordb.list_collections()

In [ ]:
# Create a new chroma collection
import chromadb
client = chromadb.PersistentClient(path=persist_directory)
collection_name = "mailContent"
collection = client.get_or_create_collection(name=collection_name)

In [ ]:
dataset=docs
from tqdm.notebook import tqdm
#!pip3 install --upgrade --quiet jupyter ipywidgets
#!conda install -n base -c conda-forge jupyterlab_widgets
#!conda install -n pyenv -c conda-forge ipywidgets

# Load the supporting evidence in batches of 1000
batch_size = 2
for i in tqdm(range(0, len(dataset), batch_size), desc="Adding documents"):
    collection.add(
        ids=[
            str(i) for i in range(i, min(i + batch_size, len(dataset)))
        ],  # IDs are just strings
        documents=dataset["support"][i : i + batch_size],
        metadatas=[
            {"type": "support"} for _ in range(i, min(i + batch_size, len(dataset)))
        ],
    )

In [ ]:
import sagemaker
from sagemaker.huggingface import (
    HuggingFaceModel    
)

role = sagemaker.get_execution_role()

hub_config = {
    'HF_MODEL_ID': 'sentence-transformers/all-MiniLM-L6-v2', # model_id from hf.co/models
    'HF_TASK': 'feature-extraction'
}

huggingface_model = HuggingFaceModel(
    env=hub_config,
    role=role,
    transformers_version="4.6", # transformers version used
    pytorch_version="1.7", # pytorch version used
    py_version="py36", # python version of the DLC
)

In [ ]:
encoder = huggingface_model.deploy(
    initial_instance_count=1,
    instance_type="ml.t2.large",
    endpoint_name="minilm-demo1"
)

In [ ]:
out = encoder.predict({"inputs": ["check embeddings", "check few more embeddings for testing "]})

In [ ]:
from typing import List

def embed_docs(docs: List[str]) -> List[List[float]]:
    out = encoder.predict({'inputs': docs})
    embeddings = np.mean(np.array(out), axis=1)
    return embeddings.tolist()

In [ ]:
import boto3

client = boto3.client('sagemaker-runtime')

import json

endpoint_name = "minilm-demo"                                       # Your endpoint name.
content_type = "application/json"                                     # The MIME type of the input data in the request body.
accept = "application/json"   
input_data = {"inputs": ["check embeddings", "check few more embeddings for testing "]}
    # The desired MIME type of the inference in the response.
payload =json.dumps(input_data)   


In [ ]:
response = client.invoke_endpoint(
    EndpointName=endpoint_name, 
    CustomAttributes="", 
    ContentType=content_type,
    Accept=accept,
    Body=payload
    )

In [ ]:
print(response)

In [ ]:
output_data = json.loads(response['Body'].read().decode())

In [ ]:
len(output_data[1][2])

In [ ]:
import pandas as pd
df = pd.read_json('s3://abb8-ml-model/dailydownload/mailContents.json')
print(df["mail_body"][1])

In [ ]:
print(df["mail_body"])

In [ ]:
!pip3 install -U  ipywidgets widgetsnbextension pandas-profiling jupyter tqdm

In [ ]:
print(embed_docs( df["mail_body"][0:1].to_list()))

In [ ]:
#!jupyter nbextension enable --py widgetsnbextension
import numpy as np
!pip install --upgrade --quiet jupyter_client ipywidgets
from ipywidgets import FloatProgress

from tqdm.notebook import tqdm

# Load the supporting evidence in batches of 1000
batch_size = 2
for i in range(0, len(df), batch_size):
    # find end of batch
    i_end = min(i+batch_size, len(df))
    collection.add(
        ids=[
            str(i) for i in range(i, min(i + batch_size, len(df)))
        ],  # IDs are just strings       
        embeddings = embed_docs( df["mail_body"][i:i_end].tolist()),
        #documents=df["mail_body"][i : i + batch_size],        
        metadatas= [{'text': text} for text in df["mail_body"][i:i_end]],
    )

In [ ]:
results = collection.query(
    query_texts='when is doctor appointment',
    n_results=5)

In [ ]:
print(results)

In [ ]:
#no vector db

In [ ]:
prompt_template = """Answer the following QUESTION based on the CONTEXT
given. If you do not know the answer and the CONTEXT doesn't
contain the answer truthfully say "I don't know".

CONTEXT:
{context}

QUESTION:
{question}

ANSWER:

"""

In [ ]:
context= df.groupby(['mail_body'])["mail_body"].agg(lambda x: ''.join(x))        
question="Can you please provide me with a list of my upcoming tasks based on the given context information?"

In [ ]:
print(context)

In [ ]:
text_input = prompt_template.replace("{context}", context.to_string()).replace("{question}", question)

In [ ]:
print(text_input)

In [ ]:
from transformers import pipeline
#%pip install -U sagemaker
import sagemaker
from sagemaker.huggingface import HuggingFaceModel


role = sagemaker.get_execution_role()
# Hub Model configuration. https://huggingface.co/models
hub = {
	'HF_MODEL_ID':'google/flan-t5-large',
	'HF_TASK':'text-generation'
}



In [ ]:
# create Hugging Face Model Class
huggingface_model = HuggingFaceModel(
	transformers_version='4.26.0',
	pytorch_version='1.13.1',
	py_version='py39',
	env=hub,
	role=role, 
)

predictor = huggingface_model.deploy(
	initial_instance_count=1, # number of instances
	instance_type='ml.g5.2xlarge' # ec2 instance type
)

In [ ]:
out = predictor.predict({"inputs": text_input})
generated_text = out[0]["generated_text"]
print(f"[Output]: {generated_text}")